# Dependencies

In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import time
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from datasets import camvid
from datasets import joint_transforms

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [4]:
CAMVID_PATH = Path('SegNet-Tutorial/CamVid')
RESULTS_PATH = Path('.results/')
WEIGHTS_PATH = Path('.weights/')
RESULTS_PATH.mkdir(exist_ok=True)
WEIGHTS_PATH.mkdir(exist_ok=True)
batch_size = 2

In [ ]:
normalize = transforms.Normalize(mean=camvid.mean, std=camvid.std)